In [7]:
import cv2
import numpy as np

In [ ]:
# Función para leer y convertir las imágenes a escala de grises
def read_images(filenames):
    """Lee las imágenes de los archivos y las convierte a escala de grises."""
    images = [cv2.imread(file) for file in filenames]
    images_grey = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    return images_grey

# Función para detectar características y descriptores usando SIFT
def detect_features(images_grey):
    """Detecta puntos clave y calcula descriptores para una lista de imágenes en escala de grises."""
    sift = cv2.SIFT_create()
    keypoints_descriptors = [sift.detectAndCompute(image, None) for image in images_grey]
    return keypoints_descriptors

# Función para emparejar descriptores entre dos imágenes
def match_descriptors(descriptor1, descriptor2):
    """Empareja descriptores entre dos conjuntos de descriptores."""
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(descriptor1, descriptor2)
    # Ordenamos los emparejamientos por su distancia (calidad del emparejamiento)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

# Función para calcular la homografía y unir las imágenes
def stitch_images(image1, image2, kp1, kp2, matches):
    """Encuentra la homografía y transforma la imagen 2 al plano de la imagen 1."""
    # Extraemos la ubicación de los puntos clave emparejados
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros_like(points1)
    
    for i, match in enumerate(matches):
        points1[i, :] = kp1[match.queryIdx].pt
        points2[i, :] = kp2[match.trainIdx].pt
    
    # Calculamos la homografía
    H, _ = cv2.findHomography(points2, points1, cv2.RANSAC, 5.0)
    
    # Transformamos las imágenes
    width = image1.shape[1] + image2.shape[1]
    height = max(image1.shape[0], image2.shape[0])
    result = cv2.warpPerspective(image2, H, (width, height))
    result[0:image1.shape[0], 0:image1.shape[1]] = image1
    return result


In [3]:
# Nombres de archivo de las imágenes de entrada
filenames = ['img2.png',
             'img1.png',
             'img3.png']

# Leemos y convertimos las imágenes a escala de grises
images_grey = read_images(filenames)

# Detectamos características
kps_des = detect_features(images_grey)

# Emparejamos los descriptores entre las imágenes
matches12 = match_descriptors(kps_des[0][1], kps_des[1][1])
matches23 = match_descriptors(kps_des[1][1], kps_des[2][1])

# Leemos las imágenes originales de nuevo para la unión
images = read_images(filenames)  # Leemos las imágenes a color para la unión

# Unimos las imágenes
result_12 = stitch_images(images[0], images[1], kps_des[0][0], kps_des[1][0], matches12)
result_123 = stitch_images(result_12, images[2], kps_des[1][0], kps_des[2][0], matches23)

# Guardamos y devolvemos el resultado
output_filename = '/mnt/data/panoramic_image.png'
cv2.imwrite(output_filename, result_123)

output_filename

'panoramic_image.png'

: 